# Russia Real Estate 2021 Classification

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.neighbors import  KNeighborsClassifier # Классификация К-Ближайших соседей от scikit-learn

from sklearn.model_selection import train_test_split #  функция разбиения на тренировочную и тестовую выборку
from sklearn.metrics import classification_report # функция scikit-learn которая считает много метрик классификации
from sklearn.model_selection import StratifiedKFold # при кросс-валидации разбиваем данные в пропорции целевой метки
from sklearn.model_selection import cross_validate # функция кросс-валидации от Scikit-learn

from sklearn.metrics import confusion_matrix # матрица ошибок от Scikit-learn

from sklearn.metrics import accuracy_score as acc # функция расчета доли правильных ответов от Scikit-learn
from sklearn.metrics import f1_score as f1 # функция расчета f1-меры от Scikit-learn


pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 20)
pd.set_option("display.precision", 4)
pd.set_option("plotting.backend", "matplotlib")

## Импорт

Для выполнения Домашнего задания №2 я решил продолжить работу с [набором данных о жилой недвижимости в России в 2021](https://www.kaggle.com/datasets/mrdaniilak/russia-real-estate-2021)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/russia_real_estate_DF.csv', delimiter=',')

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df_del = pd.read_csv('/content/drive/MyDrive/russia_real_estate_DF_del_vibros.csv', delimiter=',')

In [ ]:
df_del = pd.DataFrame(df_del)

In [ ]:
df.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,id_region
0,2021-01-01,2451300,15,31,1,30.3,0.1,56.7801,60.6994,0,2,620000.0,66
1,2021-01-01,1450000,5,5,1,33.0,6.0,44.6082,40.1384,0,0,385000.0,1
2,2021-01-01,10700000,4,13,3,85.0,12.0,55.5401,37.7251,3,0,142701.0,50
3,2021-01-01,3100000,3,5,3,82.0,9.0,44.6082,40.1384,0,0,385000.0,1
4,2021-01-01,2500000,2,3,1,30.0,9.0,44.7387,37.7137,3,2,353960.0,23


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112237 entries, 0 to 112236
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   date           112237 non-null  object 
 1   price          112237 non-null  int64  
 2   level          112237 non-null  int64  
 3   levels         112237 non-null  int64  
 4   rooms          112237 non-null  int64  
 5   area           112237 non-null  float64
 6   kitchen_area   112237 non-null  float64
 7   geo_lat        112237 non-null  float64
 8   geo_lon        112237 non-null  float64
 9   building_type  112237 non-null  object 
 10  object_type    112237 non-null  object 
 11  postal_code    112237 non-null  object 
 12  id_region      112237 non-null  object 
dtypes: float64(4), int64(4), object(5)
memory usage: 11.1+ MB


In [ ]:
df_del.head()

,date,price,level,levels,rooms,area,kitchen_area,geo_lat,geo_lon,building_type,object_type,postal_code,id_region
0,2021-01-01,2451300,15,31,1,30.3,0.1,56.7801,60.6994,0,2,620000.0,66
1,2021-01-01,10700000,4,13,3,85.0,12.0,55.5401,37.7251,3,1,142701.0,50
2,2021-01-01,1450000,5,5,2,47.0,6.0,48.5112,44.5668,2,1,400096.0,34
3,2021-01-01,2290000,2,2,2,53.2,16.0,53.1644,45.0340,5,1,440003.0,58
4,2021-01-01,3350000,1,19,1,10.1,2.0,55.7780,37.5401,3,1,123007.0,77


In [ ]:
df_del.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78079 entries, 0 to 78078
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           78079 non-null  object 
 1   price          78079 non-null  int64  
 2   level          78079 non-null  int64  
 3   levels         78079 non-null  int64  
 4   rooms          78079 non-null  int64  
 5   area           78079 non-null  float64
 6   kitchen_area   78079 non-null  float64
 7   geo_lat        78079 non-null  float64
 8   geo_lon        78079 non-null  float64
 9   building_type  78079 non-null  int64  
 10  object_type    78079 non-null  int64  
 11  postal_code    78079 non-null  float64
 12  id_region      78079 non-null  int64  
dtypes: float64(5), int64(7), object(1)
memory usage: 7.7+ MB


В качестве целевой метки для задачи классификации используем object_type - первчный или вторичный тип жилья. Для наглядности изменим значение первички с "0" на "1"

In [ ]:
df.loc[df['object_type']==0, ['object_type']] = 1
df_del.loc[df_del['object_type']==0, ['object_type']] = 1

In [ ]:
df["building_type"] = df["building_type"].astype("object")
df["object_type"] = df["object_type"].astype("object")
df["postal_code"] = df["postal_code"].astype("object")
df["id_region"] = df["id_region"].astype("object")

df_del["building_type"] = df_del["building_type"].astype("object")
df_del["object_type"] = df_del["object_type"].astype("object")
df_del["postal_code"] = df_del["postal_code"].astype("object")
df_del["id_region"] = df_del["id_region"].astype("object")

## Классификация Методом к-Ближайших Соседей (с выбросами)

In [ ]:
cat_columns = [] # определяем категиральные признаки
num_columns = [] # определяем числовые признаки

for column_name in df.columns:
    if (df[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['date', 'building_type', 'object_type', 'postal_code', 'id_region'] 
 len =  5
numerical columns:	  ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'geo_lat', 'geo_lon'] 
 len =  8


In [ ]:
df["object_type"] = df["object_type"].astype("int")

In [ ]:
X = df[num_columns].values  # берем только числовые данные
y = df["object_type"].values # для классификации целевая метрика - тип Трансмиссии
features_names = df[num_columns].columns

# разбиваем на тестовую и тренировочную
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

'\n# нормализуем \nscaler  = MinMaxScaler()\nscaler.fit(X_train) # для тренировочных сначала "обучаем"\nX_train = scaler.transform(X_train) # потом преобразуем  transform \nX_test  = scaler.transform(X_test) # для тестовых - просто transform\n'

In [ ]:
#@title Гиперпараметры к-Ближайших Соседей{ run: "auto" }
#@markdown ### количество соседей
k = 9 #@param {type:"slider", min:1, max:15, step:1}
#@markdown ### тип весов
weights = 'distance' #@param [ 'uniform', 'distance']{type:"string"}

# создаем объект класса с указанием гиперпараметров
kNN = KNeighborsClassifier(n_neighbors=k,
                           weights=weights )

# обучаем на тренировочных данных
kNN.fit(X_train, y_train)

# предсказываем на тестовых данных
y_predict=kNN.predict(X_test)

# выбираем метрики которые будем оценивать
# в виде словаря
scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}
# используем функцию кросс-валидации
scores = cross_validate(kNN, # какую модель тестируем
                        X_train, y_train, # на каких данных
                      scoring=scoring, #на каких метриках
                      # как разбиваем данные при кросс-валидации
                      cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_kNN = pd.DataFrame(scores)
display(DF_cv_kNN)
print('\n')
print(DF_cv_kNN.mean()[2:])

# та самая функция которая считает много метрик
print (classification_report(y_test, # правильные ответы
                             y_predict, # предсказания
                             # имена классов
                             target_names=['1','2']))

Результаты Кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,0.0874,0.1726,0.8739,0.9181,0.8823,0.9568
1,0.0901,0.1821,0.8715,0.9164,0.8817,0.9540
2,0.0881,0.1707,0.8767,0.9198,0.8844,0.9583
3,0.0908,0.1793,0.8736,0.9179,0.8818,0.9571
4,0.0882,0.1666,0.8770,0.9200,0.8855,0.9573




test_ACC          0.8745
test_F1           0.9184
test_Precision    0.8831
test_Recall       0.9567
dtype: float64
              precision    recall  f1-score   support

           1       0.89      0.96      0.92     25037
           2       0.85      0.66      0.74      8635

    accuracy                           0.88     33672
   macro avg       0.87      0.81      0.83     33672
weighted avg       0.88      0.88      0.88     33672



Сходу у модели получилось показать хорошие метрики в результате кросс-валидации. Однако попробуем найти оптимальные значения гиперпараметров методом рандамизированного поиска (RandomizedSearchCV в sklearn)

### 2. Отбор гиперпараметров рандамизированным поиском

In [ ]:
from sklearn.model_selection import RandomizedSearchCV # для поиска при случайном сэмплирование гиперпараметров
from scipy import stats

# словарь гиперпараметров в виде
# обозначение гиперпараметров : из какого распределения сэмплируем
# обозначение должно "досимвольно" совпадать с тем что в документации
parameters = {'n_neighbors':stats.randint(1,50), # задаем распределение как равномерное от 1 до 50
              'weights':['uniform', 'distance']}

# количество итераций
n_iter_search = 20

kNN_search = RandomizedSearchCV(estimator = KNeighborsClassifier(),  # оптимизируем нашу модель
                                verbose = 3,  # чтобы он всё подробно расписал. если не интересно - пишем 0
                          param_distributions=parameters , # что оптимизируем - берем из словарика
                          cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42), # указываем тип кросс-валидации
                          n_iter = n_iter_search # количество итераций
                          )

In [ ]:
kNN_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ....n_neighbors=9, weights=uniform;, score=0.848 total time=   0.6s
[CV 2/5] END ....n_neighbors=9, weights=uniform;, score=0.846 total time=   0.6s
[CV 3/5] END ....n_neighbors=9, weights=uniform;, score=0.852 total time=   0.7s
[CV 4/5] END ....n_neighbors=9, weights=uniform;, score=0.853 total time=   1.0s
[CV 5/5] END ....n_neighbors=9, weights=uniform;, score=0.853 total time=   1.0s
[CV 1/5] END ...n_neighbors=49, weights=uniform;, score=0.782 total time=   1.4s
[CV 2/5] END ...n_neighbors=49, weights=uniform;, score=0.779 total time=   2.0s
[CV 3/5] END ...n_neighbors=49, weights=uniform;, score=0.785 total time=   0.9s
[CV 4/5] END ...n_neighbors=49, weights=uniform;, score=0.787 total time=   0.7s
[CV 5/5] END ...n_neighbors=49, weights=uniform;, score=0.789 total time=   0.8s
[CV 1/5] END ..n_neighbors=10, weights=distance;, score=0.874 total time=   0.3s
[CV 2/5] END ..n_neighbors=10, weights=distance

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=KNeighborsClassifier(), n_iter=20,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2d367c1f00>,
                                        'weights': ['uniform', 'distance']},
                   verbose=3)

In [ ]:
kNN_search.best_estimator_ # выбираем лучший набор гиперпараметров для модели

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [ ]:
kNNbest=kNN_search.best_estimator_
kNNbest.fit(X_train, y_train); # обучаем модель на тренировочных данных

In [ ]:
scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}

scores = cross_validate(kNNbest, X_train, y_train,
                      scoring=scoring,
                      cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_best_kNN = pd.DataFrame(scores)
display(DF_cv_best_kNN)
print('\n')
print(DF_cv_best_kNN.mean()[2:])

Результаты Кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,0.1664,0.2974,0.8741,0.9183,0.8817,0.9581
1,0.2023,0.3274,0.8720,0.9168,0.8812,0.9554
2,0.1619,0.3085,0.8776,0.9205,0.8842,0.9598
3,0.1602,0.3069,0.8732,0.9177,0.8805,0.9583
4,0.1654,0.3144,0.8772,0.9202,0.8846,0.9587




test_ACC          0.8748
test_F1           0.9187
test_Precision    0.8825
test_Recall       0.9581
dtype: float64


Метрики модели с оптимальными гиперпараметрами слабо изменились в сравнении моделью, где значения мы задавали руками (изменения колеблются в районе 0.001 - видимо изначально хорошо попали)

In [ ]:
y_predict=kNNbest.predict(X_test)

print (classification_report(y_test, y_predict, target_names=['1','2']))

              precision    recall  f1-score   support

           1       0.89      0.96      0.92     25037
           2       0.85      0.65      0.74      8635

    accuracy                           0.88     33672
   macro avg       0.87      0.81      0.83     33672
weighted avg       0.88      0.88      0.88     33672



Результаты перебора.  
Оптимальными оказался набор: weights = ‘distance’, n_neighbors = 10


In [ ]:
DF_class=pd.DataFrame(kNN_search.cv_results_)
DF_class

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.1206,0.0328,0.6569,0.1503,9,uniform,"{'n_neighbors': 9, 'weights': 'uniform'}",0.8478,0.8457,0.8519,0.8530,0.8534,0.8503,0.0031,10
1,0.1430,0.0430,1.0099,0.4670,49,uniform,"{'n_neighbors': 49, 'weights': 'uniform'}",0.7815,0.7788,0.7850,0.7868,0.7887,0.7842,0.0036,20
2,0.0927,0.0015,0.1787,0.0097,10,distance,"{'n_neighbors': 10, 'weights': 'distance'}",0.8741,0.8720,0.8776,0.8732,0.8772,0.8748,0.0022,1
3,0.0955,0.0096,0.1968,0.0072,15,distance,"{'n_neighbors': 15, 'weights': 'distance'}",0.8709,0.8694,0.8752,0.8728,0.8758,0.8728,0.0024,2
4,0.0928,0.0034,0.4889,0.0505,2,uniform,"{'n_neighbors': 2, 'weights': 'uniform'}",0.8490,0.8525,0.8522,0.8480,0.8511,0.8505,0.0018,9
5,0.1302,0.0330,0.4402,0.1005,44,distance,"{'n_neighbors': 44, 'weights': 'distance'}",0.8618,0.8571,0.8658,0.8606,0.8657,0.8622,0.0033,7
6,0.1593,0.0027,0.2615,0.0051,4,distance,"{'n_neighbors': 4, 'weights': 'distance'}",0.8686,0.8672,0.8744,0.8668,0.8711,0.8696,0.0028,4
7,0.0962,0.0020,0.3047,0.0100,42,distance,"{'n_neighbors': 42, 'weights': 'distance'}",0.8622,0.8575,0.8658,0.8610,0.8662,0.8625,0.0032,6
8,0.0954,0.0038,0.5252,0.0271,17,uniform,"{'n_neighbors': 17, 'weights': 'uniform'}",0.8276,0.8249,0.8336,0.8294,0.8342,0.8299,0.0035,15
9,0.0965,0.0047,0.6742,0.2504,26,uniform,"{'n_neighbors': 26, 'weights': 'uniform'}",0.8112,0.8070,0.8151,0.8103,0.8132,0.8114,0.0027,17


## Классификация Методом к-Ближайших Соседей (БЕЗ выбросов)

Проделаем те же операции для набора без выбросов.

In [ ]:
cat_columns = [] # определяем категиральные признаки
num_columns = [] # определяем числовые признаки

for column_name in df_del.columns:
    if (df_del[column_name].dtypes == object):
        cat_columns +=[column_name]
    else:
        num_columns +=[column_name]

print('categorical columns:\t ',cat_columns, '\n len = ',len(cat_columns))

print('numerical columns:\t ',  num_columns, '\n len = ',len(num_columns))

categorical columns:	  ['date', 'building_type', 'object_type', 'postal_code', 'id_region'] 
 len =  5
numerical columns:	  ['price', 'level', 'levels', 'rooms', 'area', 'kitchen_area', 'geo_lat', 'geo_lon'] 
 len =  8


In [ ]:
df_del["object_type"] = df_del["object_type"].astype("int")

In [ ]:
X = df_del[num_columns].values  # берем только числовые данные
y = df_del["object_type"].values # для классификации целевая метрика - тип Трансмиссии
features_names = df_del[num_columns].columns

# разбиваем на тестовую и тренировочную
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
#@title Гиперпараметры к-Ближайших Соседей{ run: "auto" }
#@markdown ### количество соседей
k = 9 #@param {type:"slider", min:1, max:15, step:1}
#@markdown ### тип весов
weights = 'distance' #@param [ 'uniform', 'distance']{type:"string"}

# создаем объект класса с указанием гиперпараметров
kNN = KNeighborsClassifier(n_neighbors=k,
                           weights=weights )

# обучаем на тренировочных данных
kNN.fit(X_train, y_train)

# предсказываем на тестовых данных
y_predict=kNN.predict(X_test)

# выбираем метрики которые будем оценивать
# в виде словаря
scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}
# используем функцию кросс-валидации
scores = cross_validate(kNN, # какую модель тестируем
                        X_train, y_train, # на каких данных
                      scoring=scoring, #на каких метриках
                      # как разбиваем данные при кросс-валидации
                      cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_kNN = pd.DataFrame(scores)
display(DF_cv_kNN)
print('\n')
print(DF_cv_kNN.mean()[2:])

# та самая функция которая считает много метрик
print (classification_report(y_test, # правильные ответы
                             y_predict, # предсказания
                             # имена классов
                             target_names=['1','2']))

Результаты Кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,0.0556,0.1171,0.8828,0.9262,0.8910,0.9644
1,0.0591,0.1199,0.8818,0.9257,0.8893,0.9652
2,0.0562,0.1139,0.8816,0.9255,0.8904,0.9634
3,0.0554,0.1111,0.8768,0.9227,0.8853,0.9633
4,0.0562,0.1152,0.8826,0.9260,0.8918,0.9630




test_ACC          0.8811
test_F1           0.9252
test_Precision    0.8896
test_Recall       0.9639
dtype: float64
              precision    recall  f1-score   support

           1       0.89      0.97      0.93     17763
           2       0.85      0.63      0.72      5661

    accuracy                           0.88     23424
   macro avg       0.87      0.80      0.83     23424
weighted avg       0.88      0.88      0.88     23424



По сравнению с моделью, обученной на наборе данных с выбросами, текущая модель показывает небольшое улучшение метрик на кросс-валидации (в районе 0.01 для каждой)

### 2. Отбор гиперпараметров рандамизированным поиском

In [ ]:
from sklearn.model_selection import RandomizedSearchCV # для поиска при случайном сэмплирование гиперпараметров
from scipy import stats

# словарь гиперпараметров в виде
# обозначение гиперпараметров : из какого распределения сэмплируем
# обозначение должно "досимвольно" совпадать с тем что в документации
parameters = {'n_neighbors':stats.randint(1,50), # задаем распределение как равномерное от 1 до 50
              'weights':['uniform', 'distance']}

# количество итераций
n_iter_search = 20

kNN_search = RandomizedSearchCV(estimator = KNeighborsClassifier(),  # оптимизируем нашу модель
                                verbose = 3,  # чтобы он всё подробно расписал. если не интересно - пишем 0
                          param_distributions=parameters , # что оптимизируем - берем из словарика
                          cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42), # указываем тип кросс-валидации
                          n_iter = n_iter_search # количество итераций
                          )

In [ ]:
kNN_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END ...n_neighbors=28, weights=uniform;, score=0.820 total time=   0.7s
[CV 2/5] END ...n_neighbors=28, weights=uniform;, score=0.817 total time=   0.7s
[CV 3/5] END ...n_neighbors=28, weights=uniform;, score=0.817 total time=   0.7s
[CV 4/5] END ...n_neighbors=28, weights=uniform;, score=0.815 total time=   0.7s
[CV 5/5] END ...n_neighbors=28, weights=uniform;, score=0.822 total time=   0.7s
[CV 1/5] END ..n_neighbors=35, weights=distance;, score=0.871 total time=   0.3s
[CV 2/5] END ..n_neighbors=35, weights=distance;, score=0.871 total time=   0.2s
[CV 3/5] END ..n_neighbors=35, weights=distance;, score=0.870 total time=   0.2s
[CV 4/5] END ..n_neighbors=35, weights=distance;, score=0.868 total time=   0.2s
[CV 5/5] END ..n_neighbors=35, weights=distance;, score=0.872 total time=   0.2s
[CV 1/5] END ...n_neighbors=15, weights=uniform;, score=0.843 total time=   0.4s
[CV 2/5] END ...n_neighbors=15, weights=uniform

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=KNeighborsClassifier(), n_iter=20,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2d38d29d80>,
                                        'weights': ['uniform', 'distance']},
                   verbose=3)

In [ ]:
kNN_search.best_estimator_

KNeighborsClassifier(n_neighbors=8, weights='distance')

Любопытно, но для текущей модели оптимальное кол-во n_neighbors=8, а не 10.

In [ ]:
kNNbest=kNN_search.best_estimator_
kNNbest.fit(X_train, y_train);

In [ ]:
scoring = {'ACC': 'accuracy',
           'F1': 'f1',
           'Precision': 'precision',
           'Recall': 'recall'}

scores = cross_validate(kNNbest, X_train, y_train,
                      scoring=scoring,
                      cv=StratifiedKFold(n_splits=5, shuffle = True, random_state=42))


print('Результаты Кросс-валидации')
DF_cv_best_kNN = pd.DataFrame(scores)
display(DF_cv_best_kNN)
print('\n')
print(DF_cv_best_kNN.mean()[2:])

Результаты Кросс-валидации


,fit_time,score_time,test_ACC,test_F1,test_Precision,test_Recall
0,0.0595,0.1068,0.8822,0.9256,0.8926,0.9613
1,0.0555,0.1103,0.8818,0.9256,0.8907,0.9633
2,0.0545,0.1089,0.8828,0.9261,0.8924,0.9625
3,0.0552,0.1092,0.8772,0.9228,0.8868,0.9619
4,0.0562,0.1083,0.8821,0.9255,0.8932,0.9603




test_ACC          0.8812
test_F1           0.9251
test_Precision    0.8911
test_Recall       0.9619
dtype: float64


In [ ]:
y_predict=kNNbest.predict(X_test)

print (classification_report(y_test, y_predict, target_names=['1','2']))

              precision    recall  f1-score   support

           1       0.89      0.96      0.93     17763
           2       0.85      0.64      0.73      5661

    accuracy                           0.88     23424
   macro avg       0.87      0.80      0.83     23424
weighted avg       0.88      0.88      0.88     23424



Результаты перебора

In [ ]:
DF_class=pd.DataFrame(kNN_search.cv_results_)
DF_class

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.0987,0.0019,0.6154,0.0102,28,uniform,"{'n_neighbors': 28, 'weights': 'uniform'}",0.8202,0.8171,0.8172,0.8150,0.8220,0.8183,0.0025,17
1,0.0662,0.0161,0.1742,0.0157,35,distance,"{'n_neighbors': 35, 'weights': 'distance'}",0.8706,0.8706,0.8698,0.8675,0.8722,0.8701,0.0015,6
2,0.0726,0.0199,0.4824,0.1139,15,uniform,"{'n_neighbors': 15, 'weights': 'uniform'}",0.8428,0.8446,0.8409,0.8411,0.8465,0.8432,0.0021,13
3,0.0561,0.0014,0.3372,0.0055,12,uniform,"{'n_neighbors': 12, 'weights': 'uniform'}",0.8454,0.8451,0.8442,0.8446,0.8503,0.8459,0.0022,12
4,0.0571,0.0016,0.3554,0.0132,18,uniform,"{'n_neighbors': 18, 'weights': 'uniform'}",0.8353,0.8312,0.8302,0.8330,0.8354,0.8330,0.0021,14
5,0.0580,0.0024,0.4105,0.0261,41,uniform,"{'n_neighbors': 41, 'weights': 'uniform'}",0.8091,0.8033,0.8081,0.8019,0.8045,0.8054,0.0027,19
6,0.0976,0.0017,0.5371,0.0122,5,uniform,"{'n_neighbors': 5, 'weights': 'uniform'}",0.8689,0.8649,0.8650,0.8641,0.8648,0.8655,0.0017,9
7,0.0759,0.0245,0.4332,0.0868,19,uniform,"{'n_neighbors': 19, 'weights': 'uniform'}",0.8347,0.8321,0.8298,0.8321,0.8355,0.8328,0.0021,15
8,0.0562,0.0009,0.0876,0.0030,4,distance,"{'n_neighbors': 4, 'weights': 'distance'}",0.8773,0.8792,0.8795,0.8759,0.8761,0.8776,0.0015,2
9,0.0567,0.0014,0.0855,0.0024,3,distance,"{'n_neighbors': 3, 'weights': 'distance'}",0.8762,0.8797,0.8764,0.8733,0.8761,0.8764,0.0020,3


Для модели KNeighborsClassifier нет встроенной оценки важности признаков, поскольку она не использует их для построения модели.л